**Ex N0 3 : Identification of Differentially Expressed Genes from Affymatrix Data**



In this exercise we will identify differentially expressed genes from the GEO data 'GSE20986'. You are requested to gather further information about the experiment from the GEO page & corresponding publications. The code in this tutorial is applicable for other GEO data sets as well with minimum changes. 

We will use the GEOparse library for parsing the GEO data

Install GEOparse with the following command

In [1]:
! pip install GEOparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 278 kB 4.2 MB/s 
  Created wheel for GEOparse: filename=GEOparse-2.0.3-py3-none-any.whl size=29065 sha256=d054376dafc8842aaebb44491a3310b95f7f591d01a9ea35bb1a778a02bb3d6b
  Stored in directory: /root/.cache/pip/wheels/4d/15/e8/fbf3b47444215d9728c20d7b35436b50086aa67c2ad6dcedad
Successfully built GEOparse


In [3]:
#Import all necessary libraries
import GEOparse
import pandas as pd
import pylab as pl
import seaborn as sns
import numpy as np
import networkx as nx
import scipy
import json
import itertools

In [4]:
#Download GEO data
gse = GEOparse.get_GEO(geo="GSE20986")

13-Sep-2022 07:58:42 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
13-Sep-2022 07:58:42 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE20nnn/GSE20986/soft/GSE20986_family.soft.gz to ./GSE20986_family.soft.gz
INFO:GEOparse:Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE20nnn/GSE20986/soft/GSE20986_family.soft.gz to ./GSE20986_family.soft.gz
100%|██████████| 15.5M/15.5M [00:03<00:00, 4.97MB/s]
13-Sep-2022 07:58:46 DEBUG downloader - Size validation passed
DEBUG:GEOparse:Size validation passed
13-Sep-2022 07:58:46 DEBUG downloader - Moving /tmp/tmpw7x3m606 to /content/GSE20986_family.soft.gz
DEBUG:GEOparse:Moving /tmp/tmpw7x3m606 to /content/GSE20986_family.soft.gz
13-Sep-2022 07:58:46 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE20nnn/GSE20986/soft/GSE20986_family.soft.gz
DEBUG:GEOparse:Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE20nnn/GSE20

In [5]:
def filter_genes(gse_obj, cut_off, log2, condition_list):
  if log2:
    pivoted_samples = gse_obj.pivot_samples('VALUE')[condition_list]
    return pivoted_samples
  else:
    pivoted_samples = np.log2(gse_obj.pivot_samples('VALUE')[condition_list])
  pivoted_samples_average = pivoted_samples.median(axis=1)
  print("Number of probes before filtering: ", len(pivoted_samples_average))
  expression_threshold = pivoted_samples_average.quantile(cut_off)
  expressed_probes = pivoted_samples_average[pivoted_samples_average >= expression_threshold]
  samples = np.log2(gse.pivot_samples("VALUE").loc[expressed_probes.keys()])
  samples = samples[condition_list]
  print("Number of probes after filtering: ", len(samples))
  return samples

In [6]:
def get_ttest(control_samples, post_samples):
  ttest_result = scipy.stats.ttest_ind(control_samples, post_samples, axis=1)
  ttest = pd.DataFrame({"stat": ttest_result[0], "pvalue": ttest_result[1]}, index=control_samples.index)

  return ttest

In [7]:
sample_ids = sorted([key for key in gse.gsms.keys()])
sample_ids

['GSM524662',
 'GSM524663',
 'GSM524664',
 'GSM524665',
 'GSM524666',
 'GSM524667',
 'GSM524668',
 'GSM524669',
 'GSM524670',
 'GSM524671',
 'GSM524672',
 'GSM524673']

In [8]:
iris = ['GSM524662','GSM524665','GSM524667']
retina= ['GSM524663','GSM524664','GSM524666']	
choroid=sample_ids[6:9]
huvec=sample_ids[9:12]
#iris
#retina
#choroid
huvec	

['GSM524671', 'GSM524672', 'GSM524673']

In [9]:
#huvec_samples = filter_genes(gse, 0.25, True, huvec)
#iris_smaples = filter_genes(gse, 0.25, True, iris)
retina_samples=filter_genes(gse,0.25,True,retina)
choroid_samples=filter_genes(gse,0.25,True,choroid)

In [10]:
#ttest_df = get_ttest(huvec_samples, iris_smaples)
ttest_df = get_ttest(choroid_samples, retina_samples)

In [11]:
ttest_df

,stat,pvalue
ID_REF,,
1007_s_at,-4.803976,0.008623
1053_at,4.503977,0.010790
117_at,-0.279035,0.794051
121_at,1.051130,0.352507
1255_g_at,NaN,NaN
...,...,...
AFFX-r2-Ec-bioC-5_at,2.599040,0.060109
AFFX-r2-Ec-bioD-3_at,-0.225006,0.833002
AFFX-r2-Ec-bioD-5_at,-0.384677,0.720054


In [12]:
from statsmodels.stats import multitest

In [13]:
def get_FDR(ttest_df):
  corrected_pvalue = multitest.multipletests(pvals=ttest_df['pvalue'], method='bonferroni', alpha=0.05)
  print(corrected_pvalue)
  FDR = pd.DataFrame({'Rejected': corrected_pvalue[0], 'FDR': corrected_pvalue[1]}, index= ttest_df.index)

  return FDR

In [14]:
FDR = get_FDR(ttest_df)

(array([False, False, False, ..., False, False, False]), array([1., 1., 1., ..., 1., 1., 1.]), 9.381485199799755e-07, 9.144947416552355e-07)


In [15]:
selected = FDR.loc[FDR['FDR'] < 0.1]
selected

,Rejected,FDR
ID_REF,,
219273_at,False,0.08967


In [16]:
def get_selected_df(gse_obj, selected_FDR):
  pivoted_samples = gse_obj.pivot_samples('VALUE').loc[selected_FDR.index]
  return pivoted_samples

In [17]:
selected_df = get_selected_df(gse, selected)
selected_df

name,GSM524662,GSM524663,GSM524664,GSM524665,GSM524666,GSM524667,GSM524668,GSM524669,GSM524670,GSM524671,GSM524672,GSM524673
ID_REF,,,,,,,,,,,,
219273_at,5.263468,2.495718,2.618995,5.055964,2.506239,5.391078,6.39257,6.311714,6.580927,2.606663,2.56851,3.103787


In [18]:
selected_df = np.log2(selected_df)
selected_df

name,GSM524662,GSM524663,GSM524664,GSM524665,GSM524666,GSM524667,GSM524668,GSM524669,GSM524670,GSM524671,GSM524672,GSM524673
ID_REF,,,,,,,,,,,,
219273_at,2.396014,1.319455,1.389014,2.337986,1.325524,2.430574,2.676396,2.658032,2.718291,1.382204,1.360932,1.634029


In [19]:
def get_lfc(control_samples, post_samples):
  control_mean = control_samples.mean(axis=1)
  post_mean = post_samples.mean(axis=1)
  LFCs = pd.DataFrame({"LFC": (post_mean - control_mean).to_dict()})
  return LFCs

In [20]:
huvec_samples = selected_df[huvec]
choroid_samples = selected_df[choroid]
LFCs = get_lfc(huvec_samples, choroid_samples)
LFCs

,LFC
219273_at,1.225185


In [21]:
def get_annotation(gene_dataframe, data_flatform, leftkey):
  gene_annotated = gene_dataframe.reset_index().merge(gse.gpls[data_flatform].table[["ID", "ENTREZ_GENE_ID", "Gene Symbol"]],
                                left_on=leftkey, right_on="ID").set_index(leftkey)
  del gene_annotated["ID"]
  # remove probes without ENTREZ
  gene_annotated = gene_annotated.dropna(subset=["ENTREZ_GENE_ID"])
  # remove probes with more than one gene assigned
  gene_annotated['ENTREZ_GENE_ID'] = pd.to_numeric(gene_annotated['ENTREZ_GENE_ID'], errors="coerce")
  gene_annotated.dropna(how="any", inplace=True)
  gene_annotated['ENTREZ_GENE_ID'] = gene_annotated.ENTREZ_GENE_ID.astype('int').astype('str')
  # for each gene average LFC over probes
  gene_annotated = gene_annotated.groupby("Gene Symbol").median()

  return gene_annotated

In [22]:
LFCs_annotated = get_annotation(LFCs, 'GPL570', 'index')

In [23]:
LFCs_annotated

,LFC
Gene Symbol,
CCNK,1.225185
